# NAME: RAJIB REGMI
# ID: 2438399

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer

In [ ]:
dataset_path = "/content/drive/MyDrive/Data/data (1).csv"
df = pd.read_csv(dataset_path)

In [ ]:
print(df.info())  # Get info about the dataset
print(df.describe())  # Statistical summary
print(df.isnull().sum())

#

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [ ]:
df.fillna(df.mean(numeric_only=True), inplace=True)

In [ ]:
target_column = "diagnosis"  # Ensure this is the correct target column name
X = df.drop(columns=[target_column])

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df[target_column])

In [ ]:
print(X.isnull().sum())

id                           0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed: 32                569
dtype: i

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
# Remove columns with no useful data (e.g., 'Unnamed: 32')
X_cleaned = X.drop(columns=['Unnamed: 32'], errors='ignore')  # Ignore if column doesn't exist

# Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_cleaned)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Check if there are any NaNs or infinite values in X_scaled
print("Check if there are any NaNs in X_scaled:", np.isnan(X_scaled).sum())  # Should be 0
print("Check if there are any infinite values in X_scaled:", np.isinf(X_scaled).sum())  # Should be 0

# Ensure X and y are aligned
print("Shapes of X_scaled and y:", X_scaled.shape, y.shape)  # Should match in number of rows

# Feature selection using RFE
log_reg = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=log_reg, n_features_to_select=10)
X_selected = rfe.fit_transform(X_scaled, y)

# Get the selected feature names
selected_features = X_cleaned.columns[rfe.support_]  # Use cleaned X (without 'Unnamed: 32')
print("Selected Features:", selected_features)


Check if there are any NaNs in X_scaled: 0
Check if there are any infinite values in X_scaled: 0
Shapes of X_scaled and y: (569, 31) (569,)
Selected Features: Index(['concave points_mean', 'radius_se', 'area_se', 'compactness_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'concavity_worst', 'concave points_worst'],
      dtype='object')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

In [ ]:
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y_pred = log_reg.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

Accuracy: 0.9824561403508771
Precision: 0.9838709677419355
Recall: 0.9682539682539683
F1 Score: 0.976
ROC AUC Score: 0.9794973544973545
